In [1]:
import duckdb
import re
import csv
import sys
import ahocorasick
import seaborn as sns
import os

csv.field_size_limit(sys.maxsize)
import pandas as pd
from Levenshtein import ratio


def generate_corasick(kws: list[str]):
	automation = ahocorasick.Automaton()
	for idx, key in enumerate(kws):
		automation.add_word(key, (idx, key))
	automation.make_automaton()
	return automation


def extract_keywords(text: str, corasick_auto):
	kw = set()
	for end_index, (insert_order, original_value) in corasick_auto.iter(str(text).lower()):
		kw.add(original_value)
	return list(kw) if kw else None

In [ ]:
con = duckdb.connect('../mining/result/models.db')

tables = [
	"models",
	"hf_discussions",
	"hf_discussion_events",
	"gh_repositories",
	"gh_discussions",
	"gh_comments",
	"gh_issues",
]

for table in tables:
	# df = con.execute(f"SELECT * from {table}").df()
	# df.to_csv(f"./db_dump_csv/{table}.csv", index=False)

	count = con.execute(f"SELECT COUNT(1) from {table}")
	print(f"{table=}, {count.fetchone()=}")

# con.execute(f"COPY {table} TO './db_dump_csv/{table}.csv' (HEADER, DELIMITER ',')")


In [ ]:
df = pd.read_csv("./db_dump_csv/models.csv")

df["github_links_set"] = None
df["github_links_score"] = None
df["highest_score_link"] = None
df["highest_score"] = None

# file_path = "huggingface_models_likes_all.csv"
# df = pd.read_csv(file_path, )
# df.columns = [
#     "model_id", "downloads", "downloads_all_time", "likes", "trending_score", "pipeline_tags",
#     "tags", "card_data", "base_model_from_card_data", "scan_done", "files_with_issues",
#     "adapter_count", "merge_count", "quantized_count", "finetune_count"
# ]

print(df.info())
print(len(df))

df.drop_duplicates(subset=["model_id"], inplace=True)
print(len(df))

df.dropna(subset=["github_links"], inplace=True)
print(len(df))

to_remove = [
	")", "(", ",",
]

for index, row in df.iterrows():
	print(index, row["model_id"])
	link_str = row["github_links"]
	# df["github_links_set"] = df["github_links"].apply(extract_github_repo_link_set)
	# def extract_github_repo_link_set(link_str: str):
	if pd.isna(link_str) or link_str == "" or link_str == "[]":
		df.at[index, "github_links_set"] = None
		continue

	links = link_str.split(",")
	repo_link = None
	processed_links = set()
	# authors = set()
	# repos = set()

	for link in links:
		if link == "":
			continue
		link = link.strip()
		# remove special characters
		for char in to_remove:
			link = link.replace(char, "")

		# capture entire link
		if re.search(r"https?:\/\/?github\.com\/[\w-]+\/[\w-]+", link):
			repo_link = re.search(r"https?:\/\/?github\.com\/[\w-]+\/[\w-]+", link).group(0)
			processed_links.add(repo_link)
			author_repo = repo_link.replace("https://github.com/", "")

	# capture each group
	# author, repo = re.search(r"https?:\/\/?github\.com\/([\w-]+)\/([\w-]+)", link).groups()
	# authors.add(author)
	# repos.add(repo)

	if processed_links:
		processed_links = list(processed_links)
		df.at[index, "github_links_set"] = processed_links
		scores = [ratio(link, row["model_id"]) for link in processed_links]
		df.at[index, "github_links_score"] = scores
		df.at[index, "highest_score_link"] = processed_links[scores.index(max(scores))]
		df.at[index, "highest_score"] = max(scores)
# else:
# 	df.at[index, "github_links_set"] = None
# 	df.at[index, "github_links_score"] = None

# print(list(processed_links))
# row["github_author"] = authors if authors else None
# row["github_repos"] = repos if repos else None

df.to_csv("./filtered/hf_models_with_scored_link.csv", index=False)

df.dropna(subset=["highest_score"], inplace=True)
print(len(df))
print(df.highest_score_link.nunique())
df.to_csv("./filtered/hf_models.csv", index=False)

In [ ]:
# print(len(df))
# df.drop_duplicates(subset=["model_id"], inplace=True)
# print(len(df))

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.hist(df['highest_score'].dropna(), bins=10, color='skyblue', edgecolor='black')
plt.title('Distribution of Highest Scores')
plt.xlabel('Score')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# filter other files
files = [
	"gh_comments.csv",
	"gh_discussions.csv",
	"gh_issues.csv",
	"gh_repositories.csv",
	"hf_discussion_events.csv",
	"hf_discussions.csv",
]

bots = [
	"allcontributors[bot]",
	"allstar-app[bot]",
	"azure-pipelines[bot]",
	"codeant-ai[bot]",
	"coderabbitai[bot]",
	"copybara-service[bot]",
	"dagshub[bot]",
	"deepsource-autofix[bot]",
	"dependabot-preview[bot]",
	"dependabot[bot]",
	"devin-ai-integration[bot]",
	"ellipsis-dev[bot]",
	"github-actions[bot]",
	"google-allstar-prod[bot]",
	"greenkeeper[bot]",
	"imgbot[bot]",
	"learn-build-service-prod[bot]",
	"lgtm-com[bot]",
	"linear[bot]",
	"lumberbot-app[bot]",
	"mend-for-github-com[bot]",
	"mergify[bot]",
	"microsoft-github-operations[bot]",
	"microsoft-github-policy-service[bot]",
	"opensearch-trigger-bot[bot]",
	"pre-commit-ci[bot]",
	"pull[bot]",
	"pytorch-bot[bot]",
	"renovate[bot]",
	"restyled-io[bot]",
	"sentry-io[bot]",
	"sourcery-ai[bot]",
	"stainless-app[bot]",
	"sweep-ai[bot]",
	"sweep-nightly[bot]",
	"sync-by-unito[bot]",
	"vs-code-engineering[bot]",
]

for file in files:
	# needs to remove bot issues
	df = pd.read_csv(f"./db_dump_csv/{file}")
	print(f"{file} original count {len(df)}")

	if "author_login" in list(df):
		df = df[~df['author_login'].isin(bots)]
	elif "user_login" in list(df):
		df = df[~df['user_login'].isin(bots)]

	df.drop_duplicates()

	print(f"{file} filtered count {len(df)}")
	df.to_csv(f"./filtered/{file}", index=False)

In [ ]:
kws1 = ["vulnerability", "vulnerabilities", "cwe", "CWE", "cve", "CVE", "security"]

corasick = generate_corasick(kws1)

# df = pd.read_csv("./result/hf/hf_models_commits.csv")
# df["keywords"] = None
# for index, row in df.iterrows():
#     df.at[index, 'keywords_title'] = extract_keywords(row['title'], corasick)
#     df.at[index, 'keywords_message'] = extract_keywords(row['message'], corasick)
# print(f"Record with KW founds: {len(df[df['keywords_message'].notna()])} in {len(df)}")
# df[(df['keywords_title'].notna()) | (df['keywords_message'].notna())].to_csv(f"./result/hf/hf_commits_kws1.csv",
#                                                                              index=False)

# Find kws in HF

df = pd.read_csv("./filtered/hf_discussions.csv")
df["keywords"] = None
for index, row in df.iterrows():
	df.at[index, 'keywords'] = extract_keywords(row['title'], corasick)
print(f"HF Discussions with KW founds: {len(df[df['keywords'].notna()])} in {len(df)}")
df[df['keywords'].notna()].to_csv(f"./filtered_with_kw/hf_discussions.csv", index=False)

df = pd.read_csv("./filtered/hf_discussion_events.csv")
df["keywords"] = None
for index, row in df.iterrows():
	df.at[index, 'keywords'] = extract_keywords(row['content'], corasick)
print(f"HF Discussions Events with KW founds: {len(df[df['keywords'].notna()])} in {len(df)}")
df[df['keywords'].notna()].to_csv(f"./filtered_with_kw/hf_discussion_events.csv", index=False)

In [ ]:
# Find kws in GH

df = pd.read_csv("./filtered/gh_discussions.csv")
df["keywords"] = None
for index, row in df.iterrows():
	df.at[index, 'keywords'] = extract_keywords(
		str(row['discussion_title']) + "\n" + str(row['discussion_body']),
		corasick
	)
print(f"GH Discussions KW founds: {len(df[df['keywords'].notna()])} in {len(df)}")
df[df['keywords'].notna()].to_csv(f"./filtered_with_kw/gh_discussions.csv", index=False)

df = pd.read_csv("./filtered/gh_comments.csv")
df["keywords"] = None
for index, row in df.iterrows():
	df.at[index, 'keywords'] = extract_keywords(row['comment_body'], corasick)
print(f"GH Comments KW founds: {len(df[df['keywords'].notna()])} in {len(df)}")
df[df['keywords'].notna()].to_csv(f"./filtered_with_kw/gh_comments.csv", index=False)

# gh_issues.csv
df = pd.read_csv("./filtered/gh_issues.csv")
df["keywords"] = None
for index, row in df.iterrows():
	df.at[index, 'keywords'] = extract_keywords(
		str(row['issue_title']) + "\n" + str(row['issue_body']),
		corasick
	)
print(f"GH Issues KW founds: {len(df[df['keywords'].notna()])} in {len(df)}")
df[df['keywords'].notna()].to_csv(f"./filtered_with_kw/gh_issues.csv", index=False)

In [ ]:
# merge from filtered HF_DISCUSSION, GH_DISCUSSION and GH_ISSUES
# keep all discussions even if they do not have any comments
hf_discussions = pd.read_csv("./filtered/hf_discussions.csv")
hf_discussion_events = pd.read_csv("./filtered/hf_discussion_events.csv")
merged_hf = pd.merge(hf_discussions, hf_discussion_events, on=["model_id", "num"], how="left")
print("HF merged ", len(merged_hf))
merged_hf.drop(columns=["keywords_x", "keywords_y"], inplace=True)
merged_hf.to_csv("./merged/merged_hf_discussions.csv", index=False)

gh_discussions = pd.read_csv("./filtered/gh_discussions.csv")
gh_comments = pd.read_csv("./filtered/gh_comments.csv")
merged_gh = pd.merge(gh_discussions, gh_comments, on=["repo_name", "discussion_number"], how="left")
print("GH merged", len(merged_gh))
merged_gh.drop(columns=["id_x", "id_y"], inplace=True)
merged_gh.to_csv("./merged/merged_gh_discussions.csv", index=False)


In [ ]:
# merge from filtered with keywords HF_DISCUSSION, GH_DISCUSSION and GH_ISSUES
def combine_keywords(row):
	keywords = set()
	if not pd.isna(row["keywords_x"]):
		kws = eval(row["keywords_x"])
		for kw in kws:
			keywords.add(kw)
	if not pd.isna(row["keywords_y"]):
		kws = eval(row["keywords_x"])
		for kw in kws:
			keywords.add(kw)
	if keywords:  # Check if the list is not empty
		return list(keywords)
	else:
		return None


hf_discussions = pd.read_csv("./filtered_with_kw/hf_discussions.csv")
hf_discussion_events = pd.read_csv("./filtered_with_kw/hf_discussion_events.csv")
merged_hf = pd.merge(hf_discussions, hf_discussion_events, on=["model_id", "num"], how="left")
merged_hf.drop_duplicates(inplace=True)
merged_hf["keywords"] = merged_hf.apply(combine_keywords, axis=1)
merged_hf.drop(columns=["keywords_x", "keywords_y"], inplace=True)
print(len(merged_hf))
merged_hf.to_csv("./merged_with_kw/merged_hf_discussions.csv", index=False)

gh_discussions = pd.read_csv("./filtered_with_kw/gh_discussions.csv")
gh_comments = pd.read_csv("./filtered_with_kw/gh_comments.csv")
merged_gh = pd.merge(gh_discussions, gh_comments, on=["repo_name", "discussion_number"], how="left")
merged_gh.drop_duplicates(inplace=True)
merged_gh["keywords"] = merged_gh.apply(combine_keywords, axis=1)
merged_gh.drop(columns=["keywords_x", "keywords_y", "id_x", "id_y"], inplace=True)
print("GH merged", len(merged_gh))
merged_gh.to_csv("./merged_with_kw/merged_gh_discussions.csv", index=False)

# Copy from filtered_with_kw to manual for manual labelling
# Process:
- manual label hf_discussion
- manual label hf_discussion_events
- find all discussion_event where hf_discussion is_security = 1
- reverse find all discussion where hf_discussion_event is_security = 1
=> manual dataset for HF discussion



In [ ]:
# reconstruct the hf comments links
# and set up label
df = pd.read_csv("./filtered_with_kw/hf_discussions.csv")
df["is_security"] = -1
df["security_category"] = None
df["keyword_count"] = df["keywords"].apply(lambda x: len(eval(x)) if pd.notna(x) else 0)
counts = {n: len(df[df["keyword_count"] == n]) for n in range(0, 7)}
df.drop_duplicates(inplace=True)
print("HF Discussions with n keywords:", counts)
df.to_csv("./manual/hf_discussions_working.csv", index=False)

df = pd.read_csv("./filtered_with_kw/hf_discussion_events.csv")
df["url"] = "https://huggingface.co/" + df["model_id"] + "/discussions/" + df["num"].astype(str) + "#" + df["event_id"]
df["is_security"] = -1
df["security_category"] = None
df["keyword_count"] = df["keywords"].apply(lambda x: len(eval(x)) if pd.notna(x) else 0)
counts = {n: len(df[df["keyword_count"] == n]) for n in range(0, 7)}
df.drop_duplicates(inplace=True)
print("HF Discussions Event with n keywords:", counts)
df.to_csv("./manual/hf_discussion_events_working.csv", index=False)

df = pd.read_csv("./filtered_with_kw/gh_discussions.csv")
df["url"] = "https://github.com/" + df["repo_name"] + "/discussions/" + df["discussion_number"].astype(str)
df["is_security"] = -1
df["security_category"] = None
df["keyword_count"] = df["keywords"].apply(lambda x: len(eval(x)) if pd.notna(x) else 0)
counts = {n: len(df[df["keyword_count"] == n]) for n in range(0, 7)}
df = df.drop(columns=["id"]).drop_duplicates(
	subset=["repo_name", "discussion_number", "discussion_title", "author_login"])
print("GH Discussions with n keywords:", counts)
df.to_csv("./manual/gh_discussions_working.csv", index=False)

df = pd.read_csv("./filtered_with_kw/gh_comments.csv")
df["url"] = "https://github.com/" + df["repo_name"] + "/discussions/" + df["discussion_number"].astype(str)
df["is_security"] = -1
df["security_category"] = None
df["keyword_count"] = df["keywords"].apply(lambda x: len(eval(x)) if pd.notna(x) else 0)
counts = {n: len(df[df["keyword_count"] == n]) for n in range(0, 7)}
df = df.drop(columns=["id"]).drop_duplicates()
print("GH Discussions Comment with n keywords:", counts)
df.to_csv("./manual/gh_comments_working.csv", index=False)

df = pd.read_csv("./filtered_with_kw/gh_issues.csv")
df["url"] = "https://github.com/" + df["repo_name"] + "/issues/" + df["issue_number"].astype(str)
df["is_security"] = -1
df["security_category"] = None
df["keyword_count"] = df["keywords"].apply(lambda x: len(eval(x)) if pd.notna(x) else 0)
counts = {n: len(df[df["keyword_count"] == n]) for n in [0, 1, 2, 3, 4, 5, 6]}
df = df.drop(columns=["id"]).drop_duplicates()
print("GH issues with n keywords:", counts)
df[df["keyword_count"] >= 4].to_csv("./manual/gh_issues_subset_working.csv", index=False)
df.to_csv("./manual/gh_issues_working.csv", index=False)


In [ ]:
# test get cve in gh issues
# df = pd.read_csv("./filtered_with_kw/gh_issues.csv")
# df["keyword_count"] = df["keywords"].apply(lambda x: len(eval(x)) if pd.notna(x) else 0)
# counts = {n: len(df[df["keyword_count"] == n]) for n in [0, 1, 2, 3, 4, 5, 6]}
# print("GH issues with n keywords:", counts)
# df_cve = df[df["keywords"].str.contains("cve|CVE", na=False)]
# print(len(df_cve))
# df_cve
# NOTES: cve contains a lot of false positive

df = pd.read_csv("./filtered_with_kw/gh_issues.csv")
df["url"] = "https://github.com/" + df["repo_name"] + "/issues/" + df["issue_number"].astype(str)
df["is_security"] = -1
df["security_category"] = None
df["keyword_count"] = df["keywords"].apply(lambda x: len(eval(x)) if pd.notna(x) else 0)
counts = {n: len(df[df["keyword_count"] == n]) for n in [0, 1, 2, 3, 4, 5, 6]}
df = df.drop(columns=["id"]).drop_duplicates()
print("GH issues with n keywords:", counts)
df[df["keyword_count"] >= 3].to_csv("./manual/gh_issues_subset_3_working.csv", index=False)

In [ ]:
# HF label reverse look up
hf_discussion = pd.read_csv("./manual/hf_discussions_done.csv")
hf_discussion_event = pd.read_csv("./manual/hf_discussion_events_done.csv")
merged_hf = pd.read_csv("./merged/merged_hf_discussions.csv")
print(f"{len(merged_hf)=}")
merged_hf.drop_duplicates(inplace=True)
# merged_hf.dropna(subset=["content"], inplace=True)
merged_hf["is_security"] = 0

df_non_security_disc = hf_discussion[hf_discussion["is_security"] == 0][["model_id", "num"]]
df_non_security_event = hf_discussion_event[hf_discussion_event["is_security"] == 0][["model_id", "num"]]
non_security_union = pd.concat([df_non_security_disc, df_non_security_event]).drop_duplicates()
print(f"{len(non_security_union)=}")

df_security_disc = hf_discussion[hf_discussion["is_security"] == 1][["model_id", "num"]]
df_security_event = hf_discussion_event[hf_discussion_event["is_security"] == 1][["model_id", "num"]]
security_union = pd.concat([df_security_disc, df_security_event]).drop_duplicates()
print(f"{len(security_union)=}")

# keep only the rows we labelled in manual set, either 0 or 1
merged_hf = merged_hf.merge(
	pd.concat([non_security_union, security_union]).drop_duplicates(),
	on=["model_id", "num"],
	how="inner"
)
print(f"{len(merged_hf)=}")

merged_hf["is_security"] = merged_hf[["model_id", "num"]].apply(tuple, axis=1).isin(
	pd.concat([df_security_disc, df_security_event]).apply(tuple, axis=1)
).astype(int)

merged_hf.loc[
	merged_hf[["model_id", "num"]].apply(tuple, axis=1).isin(security_union.apply(tuple, axis=1)),
	"is_security"
] = 1

merged_hf = merged_hf[merged_hf["event_type"] == "comment"]

merged_hf.to_csv("./merged_after_manual/merged_hf_discussions.csv", index=False)

merged_hf_sec = merged_hf[merged_hf["is_security"] == 1]
print(f"{len(merged_hf_sec)=}")
merged_hf_sec.to_csv("./merged_after_manual/merged_hf_discussions_security.csv", index=False)

# count the distinct combination of model_id and num
distinct_discussions = merged_hf[['model_id', 'num']].drop_duplicates().shape[0]
print(f"Distinct HF discussion: {distinct_discussions}")

distinct_discussions = merged_hf_sec[['model_id', 'num']].drop_duplicates().shape[0]
print(f"Distinct Security HF discussion: {distinct_discussions}")


In [ ]:
# GH label reverse look up
gh_discussion = pd.read_csv("./manual/gh_discussions_done.csv")
gh_comments = pd.read_csv("./manual/gh_comments_done.csv")
merged_gh = pd.read_csv("./merged/merged_gh_discussions.csv")
print(f"{len(merged_gh)=}")
merged_gh.drop_duplicates(inplace=True)
merged_gh["is_security"] = 0

df_non_security_disc = gh_discussion[gh_discussion["is_security"] == 0][["repo_name", "discussion_number"]]
print(f"{len(df_non_security_disc)=}")
df_non_security_cmt = gh_comments[gh_comments["is_security"] == 0][["repo_name", "discussion_number"]]
print(f"{len(df_non_security_cmt)=}")
non_security_union = pd.concat([df_non_security_disc, df_non_security_cmt]).drop_duplicates()
print(f"{len(non_security_union)=}")

df_security_disc = gh_discussion[gh_discussion["is_security"] == 1][["repo_name", "discussion_number"]]
print(f"{len(df_security_disc)=}")
df_security_cmt = gh_comments[gh_comments["is_security"] == 1][["repo_name", "discussion_number"]]
print(f"{len(df_security_cmt)=}")
security_union = pd.concat([df_security_disc, df_security_cmt]).drop_duplicates()
# print(security_union)
print(f"{len(security_union)=}")

merged_gh = merged_gh.merge(
	pd.concat([non_security_union, security_union]).drop_duplicates(),
	on=["repo_name", "discussion_number"],
	how="inner"
)
print(f"{len(merged_gh)=}")

merged_gh.loc[
	merged_gh[["repo_name", "discussion_number"]].apply(tuple, axis=1).isin(security_union.apply(tuple, axis=1)),
	"is_security"
] = 1
print(f"{len(merged_gh)=}")
merged_gh.to_csv("./merged_after_manual/merged_gh_discussions.csv", index=False)

merged_gh_sec = merged_gh[merged_gh["is_security"] == 1]
print(f"{len(merged_gh_sec)=}")
merged_gh_sec.to_csv("./merged_after_manual/merged_gh_discussions_security.csv", index=False)

# count the distinct combination of model_id and num
distinct_discussions = merged_gh[['repo_name', 'discussion_number']].drop_duplicates().shape[0]
print(f"Distinct GH discussion: {distinct_discussions}")

distinct_discussions = merged_gh_sec[['repo_name', 'discussion_number']].drop_duplicates().shape[0]
print(f"Distinct Security HF discussion: {distinct_discussions}")

# More manual check here on the before the final dataset

In [ ]:
# manual + data from external
merged_gh = pd.read_csv("./merged_after_manual/merged_gh_discussions.csv")
merged_hf = pd.read_csv("./merged_after_manual/merged_hf_discussions.csv")
issues = pd.read_csv("./manual/gh_issues_subset_3_done.csv")
issues_external_sec = pd.read_csv("./external_issues/github_sec_issues.csv", delimiter=";")
issues_external_non_sec = pd.read_csv("./external_issues/github_nonsec_issues.csv", delimiter=";")

# repo_name,discussion_number,discussion_title,discussion_body,author_login_x,author_login_y,comment_body,is_security
merged_columns = [
	"id_name", "id_num", "type", "content", "is_security"
]

# all_df = pd.DataFrame(columns=merged_columns)
merged_gh["id_name"] = merged_gh["repo_name"]
merged_gh["id_num"] = merged_gh["discussion_number"]
merged_gh["content"] = (
	merged_gh["discussion_title"].fillna("").str.strip() + " " +
	merged_gh["discussion_body"].fillna("").str.strip() + " " +
	merged_gh["comment_body"].fillna("").str.strip()
).str.strip()
merged_gh["type"] = "GH_DISCUSSIONS"

# model_id,num,title,git_ref,url,event_id,event_type,content,is_security
merged_hf["id_name"] = merged_hf["model_id"]
merged_hf["id_num"] = merged_hf["num"]
merged_hf["content"] = (
	merged_hf["title"].fillna("").str.strip() + " " +
	merged_hf["content"].fillna("").str.strip()
).str.strip()
merged_hf["type"] = "HF_DISCUSSIONS"

# repo_name,issue_url,pr_from_issue,user_login,issue_number,keywords,url,issue_title,issue_body,is_security,security_category,keyword_count
issues["id_name"] = issues["repo_name"]
issues["id_num"] = issues["issue_number"]
issues["content"] = (
	issues["issue_title"].fillna("").str.strip() + " " +
	issues["issue_body"].fillna("").str.strip()
)
issues["type"] = "GH_ISSUES"

all_df = pd.concat(
	[
		merged_gh[merged_columns],
		merged_hf[merged_columns],
		issues[merged_columns]
	]
)
print("All manual records", len(all_df))
# shuffle
all_df = all_df.sample(frac=1)
all_df.to_csv("./merged_after_manual/merged_all.csv", index=False)

issues_external_sec["id_name"] = issues_external_sec["repository"]
issues_external_sec["id_num"] = issues_external_sec.apply(lambda x: int(str(x["issue_api_url"]).strip().split("/")[-1]),
														  axis=1)
issues_external_sec["content"] = (
	issues_external_sec["issue_title"].fillna("").str.strip() + " " +
	issues_external_sec["description"].fillna("").str.strip()
)
issues_external_sec["type"] = "GH_ISSUES_EXTERNAL"
issues_external_sec["is_security"] = 1

issues_external_non_sec["id_name"] = issues_external_non_sec["repository"]
issues_external_non_sec["id_num"] = issues_external_non_sec.apply(
	lambda x: int(str(x["issue_api_url"]).strip().split("/")[-1]), axis=1)
issues_external_non_sec["content"] = (
	issues_external_non_sec["issue_title"].fillna("").str.strip() + " " +
	issues_external_non_sec["description"].fillna("").str.strip()
)
issues_external_non_sec["type"] = "GH_ISSUES_EXTERNAL"
issues_external_non_sec["is_security"] = 0

all_df = pd.concat(
	[
		all_df,
		issues_external_sec[merged_columns],
		issues_external_non_sec[merged_columns]
	]
)
print("Manual records + external issues", len(all_df))
# shuffle
all_df = all_df.sample(frac=1)
all_df.to_csv("./merged_after_manual/merged_all_with_external.csv", index=False)

In [ ]:
all_df

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold

# test stratify
train_df, temp_df = train_test_split(
	all_df,
	test_size=0.2,
	random_state=42,
	stratify=all_df[["type", "is_security"]]
)
print(len(train_df[train_df["type"] == "GH_DISCUSSIONS"]))
print(len(temp_df[temp_df["type"] == "GH_DISCUSSIONS"]))
print(len(train_df[train_df["type"] == "GH_ISSUES"]))
print(len(temp_df[temp_df["type"] == "GH_ISSUES"]))

print(len(train_df[train_df["is_security"] == 1]))
print(len(temp_df[temp_df["is_security"] == 1]))

skf = StratifiedKFold(n_splits=5)
for i, (train_index, test_index) in enumerate(skf.split(train_df, train_df["is_security"])):
	print(f"Fold {i}:")
	print(f"  Train: index={train_index}")
	print(f"  Test:  index={test_index}")


# Multiple HF model can link to the same GH repo
-> We consider all of these related to 1 AI project, for example:
- meta-llama/Llama-2-7b
- meta-llama/Llama-2-7b-chat-hf
- meta-llama/Llama-2-70b-chat-hf
- meta-llama/Llama-2-7b-hf
- meta-llama/Llama-2-13b-chat-hf
- meta-llama/Llama-2-70b-hf
- meta-llama/Llama-2-13b-hf
- meta-llama/Llama-2-7b-chat
- meta-llama/Llama-2-70b
- meta-llama/Llama-2-70b-chat
- meta-llama/Llama-2-13b
- meta-llama/Llama-2-13b-chat

- All of these models link to http://github.com/facebookresearch/llama: -> belong to 1 project -> facebookresearch/llama.

In [ ]:
# run model in classifier

In [ ]:
def visualize_prob_sigmoid_distribution(file_path):
	try:
		df = pd.read_csv(file_path)

		if 'prob_sigmoid' not in df.columns:
			print(f"Error: 'prob_sigmoid' column not found in {file_path}")
			return

		if not all(0 <= x <= 1 for x in df['prob_sigmoid']):
			print("Warning: data points found out of 0-1 range. will try to filter...")
			df = df[(df['prob_sigmoid'] >= 0) & (df['prob_sigmoid'] <= 1)]
			print(f"Filtered {sum(not (0 <= x <= 1) for x in df['prob_sigmoid'])} out of range datapoints.")

		plt.figure(figsize=(10, 6))  # Adjust figure size as needed
		sns.histplot(df['prob_sigmoid'], kde=True, bins=30, color='skyblue')  #Histogram
		plt.title('Distribution of prob_sigmoid')
		plt.xlabel('prob_sigmoid')
		plt.ylabel('Frequency')
		plt.grid(axis='y', alpha=0.75)

		mean = df['prob_sigmoid'].mean()
		median = df['prob_sigmoid'].median()
		std = df['prob_sigmoid'].std()
		plt.axvline(mean, color='red', linestyle='dashed', linewidth=1, label=f'Mean: {mean:.2f}')
		plt.axvline(median, color='green', linestyle='dashed', linewidth=1, label=f'Median: {median:.2f}')
		plt.legend()

		plt.tight_layout()
		plt.show()

	except FileNotFoundError:
		print(f"Error: File not found at {file_path}")
	except pd.errors.EmptyDataError:
		print(f"Error: File {file_path} is empty.")
	except pd.errors.ParserError:
		print(f"Error: Could not parse {file_path}. Check file format.")
	except Exception as e:
		print(f"An unexpected error occurred: {e}")


file_path = 'inference/all_gh_bert_gh_last/raw_predictions.csv'
visualize_prob_sigmoid_distribution(file_path)

In [ ]:
df = pd.read_csv("inference/all_hf_bert_hf_best/raw_predictions.csv")
df_security = df[df["is_security_prediction"] == 1]
print(len(df_security))

file_path = 'inference/all_hf_bert_hf_best/raw_predictions.csv'
visualize_prob_sigmoid_distribution(file_path)


In [2]:
import os


# prediction result collection
def collect_prediction(path: str, exclude: list[str]):
	models = ["bert_base", "distilbert", "securebert", "roberta_base", "secbert", "secroberta", ]
	all_metrics_data = []

	subfolders = [
		f.path
		for f in os.scandir(path)
		if f.is_dir() and f.name not in exclude
	]

	for subfolder in subfolders:
		metrics_file_path = os.path.join(subfolder, "metrics.csv")
		if not os.path.exists(metrics_file_path):
			continue
		df = pd.read_csv(metrics_file_path)
		# only get the test result
		df = df.tail(1)
		model_type = None
		data_type = None

		match = re.search(r"_({})".format("|".join(models)), os.path.basename(subfolder))
		if match:
			model_type = str(os.path.basename(subfolder)[match.start():]).replace("_", "", 1)
			data_type = os.path.basename(subfolder)[:match.start()]

		df["input"] = data_type
		df["model_type"] = model_type
		df["subfolder"] = os.path.basename(subfolder)
		df["folder"] = path
		all_metrics_data.append(df)

	return pd.concat(all_metrics_data, ignore_index=True)


path = "./prediction"
exclude = ["backup_2402"]
metrics_predictions = collect_prediction(path, exclude)
metrics_predictions

,fold,epoch,train_loss,eval_loss,test_loss,accuracy,precision,recall,f1,f1_macro,mcc,elapsed_time,input,model_type,subfolder,folder
0,NaN,NaN,NaN,NaN,0.361670,0.950911,0.960799,0.897099,0.927857,0.945328,0.891926,15753.684496,all_external,securebert,all_external_securebert,./prediction
1,NaN,NaN,NaN,NaN,0.352707,0.975000,0.931818,0.911111,0.921348,0.953243,0.906569,518.330959,manual_gh,distilbert,manual_gh_distilbert,./prediction
2,NaN,NaN,NaN,NaN,0.180531,0.973031,0.930328,0.965957,0.947808,0.964813,0.929933,3380.589169,all,bert_base,all_bert_base,./prediction
3,NaN,NaN,NaN,NaN,0.494877,0.948718,0.947826,1.000000,0.973214,0.686607,0.486782,223.580802,manual_gh_issue,distilbert,manual_gh_issue_distilbert,./prediction
4,NaN,NaN,NaN,NaN,0.382849,0.958688,0.948107,0.933702,0.940849,0.954554,0.909175,8007.931883,all_external,distilbert,all_external_distilbert,./prediction
5,NaN,NaN,NaN,NaN,0.166664,0.986792,0.939759,0.975000,0.957055,0.974626,0.949479,1971.510443,manual_hf,roberta_base,manual_hf_roberta_base,./prediction
6,NaN,NaN,NaN,NaN,0.937315,0.888889,0.980000,0.899083,0.937799,0.708900,0.464872,237.166367,manual_gh_issue,secroberta,manual_gh_issue_secroberta,./prediction
7,NaN,NaN,NaN,NaN,0.254390,0.967638,0.939914,0.931915,0.935897,0.957126,0.914268,3458.170598,all,securebert,all_securebert,./prediction
8,NaN,NaN,NaN,NaN,0.370002,0.948238,0.910299,0.946133,0.927870,0.943753,0.887932,16331.838002,all_external,bert_base,all_external_bert_base,./prediction
9,NaN,NaN,NaN,NaN,0.133817,0.978571,0.897959,0.977778,0.936170,0.961647,0.924531,1042.465266,manual_gh,roberta_base,manual_gh_roberta_base,./prediction


In [ ]:
import json


def collect_inference(path: str, exclude: list[str]):
	models = ["llama", "deepseek", "phi4", "mistral"]
	all_metrics_data = []

	subfolders = [
		f.path
		for f in os.scandir(path)
		if f.is_dir() and f.name not in exclude
	]

	for subfolder in subfolders:
		metrics_file_path = os.path.join(subfolder, "metrics.json")
		if not os.path.exists(metrics_file_path):
			continue
		with open(metrics_file_path, 'r') as f:
			data = json.load(f)
		df = pd.DataFrame([data])
		# only get the test result
		df["subfolder"] = os.path.basename(subfolder)
		model = None
		data = None
		match = re.search(r"_(llama\d*|deepseekr\d*|phi4|mistral(_small)?)", os.path.basename(subfolder))
		if match:
			model = str(os.path.basename(subfolder)[match.start():]).replace("_", "", 1)
			data = os.path.basename(subfolder)[:match.start()]
		df["input_type"] = data
		df["model"] = model
		df["path"] = path
		all_metrics_data.append(df)
	return pd.concat(all_metrics_data, ignore_index=True)


path = "./llm"
exclude = []
metrics_infer = collect_inference(path, exclude)
metrics_infer

In [ ]:
import pandas as pd
from scipy.stats import norm


def get_subset(df_security, df_manual, col_ids: list[str]):
	records_to_exclude = df_manual[col_ids].drop_duplicates()
	col_ids_str = "_".join(col_ids)
	records_to_exclude[col_ids_str] = records_to_exclude[col_ids[0]].astype(str) + "_" + records_to_exclude[
		col_ids[1]].astype(str)
	print(f"{len(records_to_exclude)=}")
	N = len(df_security)  # Population size
	Z = norm.ppf(0.975)  # Z-score for 95% confidence
	p = 0.5  # Worst-case scenario proportion
	E = 0.05  # Margin of error

	# Sample size formula
	numerator = (N * (Z ** 2) * p * (1 - p))
	denominator = ((E ** 2) * (N - 1)) + ((Z ** 2) * p * (1 - p))
	sample_size = int(numerator / denominator)
	print(f"Required sample size: {sample_size} out of {N}")

	sample_df = pd.DataFrame()
	while len(sample_df) < sample_size:
		print(f"Current {len(sample_df)=}, {sample_size-len(sample_df)=}")
		temp_sample = df_security.sample(n=(sample_size - len(sample_df)), random_state=42)
		temp_sample[col_ids_str] = temp_sample[col_ids[0]].astype(str) + "_" + temp_sample[col_ids[1]].astype(str)
		temp_sample = temp_sample[~temp_sample[col_ids_str].isin(records_to_exclude[col_ids_str])]
		sample_df = pd.concat([sample_df, temp_sample])

	print(f"Final sample {len(sample_df)=}")
	return sample_df


expr = "all_gh_bert_gh_last"
gh = pd.read_csv(f"./inference/{expr}/raw_predictions.csv")
gh_security = gh[gh["is_security_prediction"] == 1]
gh_manual = pd.read_csv(f"./merged_after_manual/merged_gh_discussions.csv")
gh_security_sampled = get_subset(gh_security, gh_manual, ["repo_name", "discussion_number"])
gh_security_sampled.to_csv(f"./sampled/{expr}_sampled.csv", index=False)